In [1]:
import os
import cv2
import numpy as np
import pandas as pd


from tensorflow.keras.layers import Conv2D 
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import numpy as np

In [2]:
IMAGE_SIZE = [128, 128]

In [3]:
train_path = 'C:\\Users\\Syed Johan Arif\\Desktop\\UM\\Reseach Project\\Plant images\\KD Farm\\trial1\\original\\train'
test_path = 'C:\\Users\\Syed Johan Arif\\Desktop\\UM\\Reseach Project\\Plant images\\KD Farm\\trial1\\original\\test'

In [40]:
train_path = 'C:\\Users\\Syed Johan Arif\\Desktop\\UM\\Reseach Project\\Plant images\\KD Farm\\trial1\\original_and_augmented\\train'
test_path = 'C:\\Users\\Syed Johan Arif\\Desktop\\UM\\Reseach Project\\Plant images\\KD Farm\\trial1\\original_and_augmented\\test'


In [20]:
train_path = 'C:\\Users\\Syed Johan Arif\\Desktop\\UM\\Reseach Project\\Plant images\\KD Farm\\trial1\\original_and_augmented_100\\train'
test_path = 'C:\\Users\\Syed Johan Arif\\Desktop\\UM\\Reseach Project\\Plant images\\KD Farm\\trial1\\original_and_augmented_100\\test'


In [18]:
train_path = 'C:\\Users\\Syed Johan Arif\\Desktop\\UM\\Reseach Project\\Plant images\\KD Farm\\trial1\\original_and_augmented_200\\train'
test_path = 'C:\\Users\\Syed Johan Arif\\Desktop\\UM\\Reseach Project\\Plant images\\KD Farm\\trial1\\original_and_augmented_200\\test'

In [19]:
train_path = 'C:\\Users\\Syed Johan Arif\\Desktop\\UM\\Reseach Project\\Plant images\\KD Farm\\trial1\\with_ganv5t1_200\\train'
test_path = 'C:\\Users\\Syed Johan Arif\\Desktop\\UM\\Reseach Project\\Plant images\\KD Farm\\trial1\\with_ganv5t1_200\\test'


In [36]:
train_path = 'C:\\Users\\Syed Johan Arif\\Desktop\\UM\\Reseach Project\\Plant images\\KD Farm\\trial1\\with_ganv5t1_300\\train'
test_path = 'C:\\Users\\Syed Johan Arif\\Desktop\\UM\\Reseach Project\\Plant images\\KD Farm\\trial1\\with_ganv5t1_300\\test'


In [ ]:
x_train=[]

for folder in os.listdir(train_path):

    sub_path=train_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(128,128))
        print(img_arr.shape)

        x_train.append(img_arr)

x_test=[]

for folder in os.listdir(test_path):

    sub_path=test_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(128,128))

        x_test.append(img_arr)


In [20]:
train_x=np.array(x_train)
test_x=np.array(x_test)

train_x=train_x/255.0
test_x=test_x/255.0

train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (128, 128),
                                            batch_size = 32,
                                            class_mode = 'sparse')

train_y=training_set.classes
test_y=test_set.classes

training_set.class_indices
train_y.shape,test_y.shape


In [ ]:
model = Sequential()
model.add(Conv2D(64, (3,3), padding="same",input_shape=(128,128,3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, (3,3), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, (3,3), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(3, activation="softmax"))

model.summary()

In [23]:
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [ ]:
# fit the model
history = model.fit(
  train_x,
  train_y,
  epochs=100,
  batch_size=32,shuffle=True
)

In [ ]:
# plotting model loss
plt.plot(history.history['loss'], label='train loss')
plt.legend()

plt.savefig('vgg-loss-rps-1.png')
plt.show()

In [ ]:
# ploting model accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.legend()

plt.savefig('vgg-acc-rps-1.png')
plt.show()

In [ ]:
model.evaluate(test_x,test_y,batch_size=32)

In [30]:
y_pred=model.predict(test_x)
y_pred=np.argmax(y_pred,axis=1)

In [ ]:
accuracy_score(y_pred,test_y)

In [ ]:
print(classification_report(y_pred,test_y))

In [ ]:
confusion_matrix(y_pred,test_y)